In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import json
import math

In [2]:
h = 620
w = 620

def PxltoCoord(x, y, zoom, cntr_lat, cntr_lon):
    parallelMultiplier = math.cos(cntr_lat * math.pi / 180)
    degreesPerPixelX = 360 / math.pow(2, zoom + 8)
    degreesPerPixelY = 360 / math.pow(2, zoom + 8) * parallelMultiplier
    pointLat = cntr_lat - degreesPerPixelY * (y - h / 2)
    pointLng = cntr_lon + degreesPerPixelX * (x - w / 2)

    return [float(pointLat), float(pointLng)]

# Because of non linear transformation (going from pixel to coordinates), function has to be solved for X and Y
def CoordToPixel(pointLat, pointLon, test_lat, test_lon, zoom):
    parallelMultiplier = math.cos(test_lat * math.pi / 180)
    degreesPerPixelX = 360 / math.pow(2, zoom + 8)
    degreesPerPixelY = 360 / math.pow(2, zoom + 8) * parallelMultiplier

    Y = (test_lat - pointLat) / degreesPerPixelY + 0.5 * h
    X = (pointLon - test_lon) / degreesPerPixelX + 0.5 * w
    return X, Y

In [3]:
def find_specific_lookup(data, search_image, template_name):
    for entry in data:
        if entry["search_image"] == search_image:
            for template in entry["templates"]:
                if template["template"] == template_name:
                    return template
    return None

In [51]:
from matplotlib import image as mpimg

#get image pairs
base_path = os.path.dirname(os.getcwd())

#label path
lbl_path = os.path.join(base_path, 'Data/labels/train_template_matching.json')

#source and query images
s_img_path = os.path.join(base_path, 'Data\\map_train\\51.998552_4.372891copern.png')
q_img_path = os.path.join(base_path, 'Data\\train_template_matching')
print("s image path: ", s_img_path)
print("q image path: ", q_img_path)

#for now source path is constant
s_img = cv2.imread(s_img_path)

x_size = s_img.shape[1]
y_size = s_img.shape[0]

boundingbox = [4.360113, 51.991117, 4.387193, 52.006785]

coord_diff_x = boundingbox[2] - boundingbox[0]
coord_diff_y = boundingbox[3] - boundingbox[1]

coord_diff_per_pixel_x = coord_diff_x / x_size
coord_diff_per_pixel_y = coord_diff_y / y_size

print("coord_diff_per_pixel_x: ", coord_diff_per_pixel_x)
print("coord_diff_per_pixel_y: ", coord_diff_per_pixel_y)


with open(lbl_path, 'r') as file:
    label = json.load(file)

# images = []
templates = []
print(q_img_path)
for file in os.listdir(q_img_path):
    if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
            q_img = cv2.imread(os.path.join(q_img_path, file))
            # images.append([q_img[:, :, :3], s_img[:,:,:3]])
            gps = find_specific_lookup(label, '51.998552_4.372891.png', file)

            templates.append((q_img[:, :, :3], gps))



s image path:  c:\Users\lucah\Documents\hackathon\CASSINI\Data\map_train\51.998552_4.372891copern.png
q image path:  c:\Users\lucah\Documents\hackathon\CASSINI\Data\train_template_matching
coord_diff_per_pixel_x:  1.11211498973305e-05
coord_diff_per_pixel_y:  6.610970464134185e-06
c:\Users\lucah\Documents\hackathon\CASSINI\Data\train_template_matching


In [42]:
def PxltoCoordCopern(x, y):
    return [boundingbox[0] + x * coord_diff_per_pixel_x, boundingbox[1] + y * coord_diff_per_pixel_y]



# Because of non linear transformation (going from pixel to coordinates), function has to be solved for X and Y
def CoordToPixelCopern(pointLat, pointLon):
    print(pointLon - boundingbox[0])
    print(pointLat - boundingbox[1])
    x_pixel = (pointLon - boundingbox[0]) / coord_diff_per_pixel_x
    y_pixel = (pointLat - boundingbox[1]) / coord_diff_per_pixel_y
    return x_pixel, y_pixel


In [43]:
def extract_features_for_templates(templates, source_image):
    feature_list = []
    label_list = []

    sift = cv2.SIFT_create()
    kp_source, des_source = sift.detectAndCompute(source_image, None)

    for template, obj in templates:
        kp_template, des_template = sift.detectAndCompute(template, None)

        gps_coords = obj['gps_coords']
        gps_pixel = CoordToPixel(gps_coords[0], gps_coords[1], 51.999080, 4.373749, 15)
        # Match features
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des_template, des_source, k=2)

        # Lowe's ratio test
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        good_matches = good_matches[:100]
        # Extract matched keypoints
        query_pts = np.float32([kp_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
        dst_pts = np.float32([kp_source[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

        # Flatten and combine features
        src_flat = query_pts.flatten()
        dst_flat = dst_pts.flatten()
        input_features = np.concatenate([src_flat, dst_flat])

        # Append to feature list
        feature_list.append(input_features)
        label_list.append(gps_pixel)  # GPS coordinates of this template image
    max_len = max(len(features) for features in feature_list)
    # max_len = 468
    padded_features = [np.pad(features, (0, max_len - len(features))) for features in feature_list]

    return np.array(padded_features), np.array(label_list)


In [19]:
def extract_keypoint(template, source_image, sift):
    kp_source, des_source = sift.detectAndCompute(source_image, None)

    kp_template, des_template = sift.detectAndCompute(template[0], None)

    gps_coords = template[1]['gps_coords']
    gps_pixel = CoordToPixel(gps_coords[0], gps_coords[1], 51.999080, 4.373749, 15)
    # Match features
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_template, des_source, k=2)

    # Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    good_matches = good_matches[:100]
    # Extract matched keypoints
    query_pts = np.float32([kp_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
    dst_pts = np.float32([kp_source[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

    return dst_pts


In [20]:
def extract_keypoints(templates, source_image):

    sift = cv2.SIFT_create()

    for template in templates:
        extract_keypoint(template, source_image, sift)

In [21]:
from sklearn import metrics
from sklearn.cluster import DBSCAN

def cluster_keypoints(keypoints_train, cluster_alg=None):

    if cluster_alg is None:
        cluster_alg = DBSCAN(eps=0.1, min_samples=8)
   
    clustering = cluster_alg.fit(keypoints_train/620)
    labels = clustering.labels_

    return labels

In [22]:
from collections import Counter

def retrieve_relevant_points(keypoints_train, labels):
    counter = Counter(list(filter(lambda x: x >= 0, labels)))
    if len(counter) == 0:
        return []
    largest_cluster = max(counter, key=counter.get)
    cluster = []
    for i in range(len(keypoints_train)):
        if labels[i] == largest_cluster:
            cluster.append(keypoints_train[i])
    return cluster

In [45]:
def get_pixel_average(coords):
    if len(coords) == 0:
        return np.array([])
    return np.average(coords, axis=0)

In [ ]:
from haversine import haversine, Unit


def run_matching(plot_graph=False, calg=None):
    if calg is None:
        calg = DBSCAN(eps=0.1, min_samples=8)
    distances = []
    # templates_selection = templates
    sift = cv2.SIFT_create()
    empty_points = 0
    distance_too_large = 0


    for i, template in enumerate(templates):
        if i % 50 == 0:
            print(f"Iteration {i}/{len(templates)}; Empty points: {empty_points}; Distance too large: {distance_too_large}")

        keys = extract_keypoint(template, s_img, sift)
        labels = cluster_keypoints(keys, cluster_alg=calg)
        points = retrieve_relevant_points(keys, labels)
        avg_points = get_pixel_average(points)
        if len(points) == 0:
            empty_points += 1

            continue

        predicted_coords = PxltoCoordCopern(avg_points[0], avg_points[1])
        actual_coords = template[1]['gps_coords']
        distance = haversine(predicted_coords, actual_coords, unit=Unit.METERS)
        if distance > 200:
            distance_too_large += 1
            actual_pixel = CoordToPixelCopern(actual_coords[0], actual_coords[1])
            if plot_graph:
                plt.scatter(keys[:, 0], keys[:, 1], c=labels)
                plt.legend()
                plt.scatter(avg_points[0], avg_points[1], c='r')
                plt.scatter(actual_pixel[0], actual_pixel[1], c='g')
                plt.show()
        else:
            distances.append(distance)

    return distances, empty_points, distance_too_large


In [65]:
distances, empty_points, distance_too_large = run_matching()
average_distance = sum(distances)/len(distances)
print(f"Average distance from pixel: {average_distance}")
print(f"Empty points: {empty_points}")
print(f"Distance too large: {distance_too_large}")
print(f"Standard deviation: {np.std(distances)}")

Iteration 0/1000; Empty points: 0; Distance too large: 0
Iteration 50/1000; Empty points: 2; Distance too large: 0
Iteration 100/1000; Empty points: 7; Distance too large: 0
Iteration 150/1000; Empty points: 11; Distance too large: 0
Iteration 200/1000; Empty points: 13; Distance too large: 0
Iteration 250/1000; Empty points: 15; Distance too large: 0
Iteration 300/1000; Empty points: 22; Distance too large: 0
Iteration 350/1000; Empty points: 25; Distance too large: 0
Iteration 400/1000; Empty points: 26; Distance too large: 0
Iteration 450/1000; Empty points: 29; Distance too large: 0
Iteration 500/1000; Empty points: 34; Distance too large: 1
Iteration 550/1000; Empty points: 35; Distance too large: 1
Iteration 600/1000; Empty points: 40; Distance too large: 1
Iteration 650/1000; Empty points: 42; Distance too large: 1
Iteration 700/1000; Empty points: 49; Distance too large: 1
Iteration 750/1000; Empty points: 52; Distance too large: 1
Iteration 800/1000; Empty points: 55; Distance